In [ ]:
!pip install scikeras

from google.colab import drive
drive.mount('/gdrive')

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

baslangic_zamani = time.time()

VERI_YOLU = '/gdrive/MyDrive/odev_dosya/Banana_Ripeness_Classification'
KATEGORILER =  ['overripe', 'ripe', 'rotten', 'unripe']
IMG_BOYUTU = 100
num_classes = len(KATEGORILER)

veri = []
etiketler = []

# Veri kümesini yükleme ve etiketleri atama
for kategori in KATEGORILER:
    yol = os.path.join(VERI_YOLU, kategori)
    sinif_numarasi = KATEGORILER.index(kategori)
    for img in os.listdir(yol):
        try:
            img_dizi = cv2.imread(os.path.join(yol, img), cv2.IMREAD_GRAYSCALE)
            img_boyutlandirilmis = cv2.resize(img_dizi, (IMG_BOYUTU, IMG_BOYUTU))
            veri.append(img_boyutlandirilmis)
            etiketler.append(sinif_numarasi)
        except Exception as e:
            print('Exception:', e)

# Verileri ve etiketleri NumPy dizilerine dönüştürme
veri = np.array(veri)
veri = np.repeat(veri[..., np.newaxis], 3, axis=-1)
etiketler = np.array(etiketler)

# Verileri eğitim ve test kümelerine bölmek
egitim_verisi, test_verisi, egitim_etiketleri, test_etiketleri = train_test_split(veri, etiketler, test_size=0.2, random_state=42)

# Eğitim verilerini eğitim ve doğrulama kümelerine bölmek
egitim_verisi, val_verisi, egitim_etiketleri, val_etiketleri = train_test_split(egitim_verisi, egitim_etiketleri, test_size=0.2, random_state=42)

# Etiketleri one-hot encode yapma
egitim_etiketleri = to_categorical(egitim_etiketleri, num_classes)
val_etiketleri = to_categorical(val_etiketleri, num_classes)
test_etiketleri = to_categorical(test_etiketleri, num_classes)

# Geri çağrıları tanımlama
callback_list = [
    keras.callbacks.ModelCheckpoint(filepath='model.h5', monitor='val_accuracy', save_best_only=True, verbose=3),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

# CNN modeli oluşturma fonksiyonu
def cnn_model_olustur(optimizer='adam', dropout_orani=0.2):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_BOYUTU, IMG_BOYUTU, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(dropout_orani)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# KerasClassifier ile modeli sarmalama
model = KerasClassifier(build_fn=cnn_model_olustur, epochs=3, batch_size=64, verbose=0)

# Hiperparametre aralıklarını tanımlama
parametre_grid = {
    'optimizer': ['adam', 'rmsprop'],
}

# GridSearchCV'nin kullanılması
grid = GridSearchCV(estimator=model, param_grid=parametre_grid, cv=2)
grid_result = grid.fit(egitim_verisi, egitim_etiketleri, validation_data=(val_verisi, val_etiketleri), callbacks=callback_list)

# Sonuçları yazdırma
print("En İyi: %f Kullanılan Parametreler: %s" % (grid_result.best_score_, grid_result.best_params_))

# En iyi parametrelerle modeli tekrar oluşturma ve eğitme
en_iyi_model = cnn_model_olustur(optimizer=grid_result.best_params_['optimizer'])
gecmis = en_iyi_model.fit(egitim_verisi, egitim_etiketleri, batch_size=64, epochs=3, validation_data=(val_verisi, val_etiketleri), callbacks=callback_list)

# Modeli değerlendirme
test_kayip, test_basari = en_iyi_model.evaluate(test_verisi, test_etiketleri, verbose=2)
print('Test kaybı:', test_kayip)
print('Test başarısı:', test_basari)

# Test verileri üzerinde tahminler yapma
tahminler = en_iyi_model.predict(test_verisi)
y_pred = np.argmax(tahminler, axis=1)
y_true = np.argmax(test_etiketleri, axis=1)

# Sınıflandırma raporunu yazdırma
print("Sınıflandırma Raporu:")
print(classification_report(y_true, y_pred))

# Karmaşıklık matrisini hesaplama
cm_model = confusion_matrix(y_true, y_pred)

# Karmaşıklık matrisini görselleştirme
etiket_isimleri = ['overripe', 'ripe', 'rotten', 'unripe']
plt.figure(figsize=(8, 6))
sns.heatmap(cm_model, annot=True, cmap='Blues', fmt='.2f', xticklabels=etiket_isimleri, yticklabels=etiket_isimleri)
plt.title('Karmaşıklık Matrisi')
plt.xlabel('Tahmin Edilen Etiket')
plt.ylabel('Gerçek Etiket')
plt.show()

# Transfer Learning ile Modeli Oluşturma
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_BOYUTU, IMG_BOYUTU, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

transfer_model = Model(inputs=base_model.input, outputs=predictions)

# Transfer Learning Modelini Derleme
transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Transfer Learning Modelini Eğitme
transfer_gecmis = transfer_model.fit(egitim_verisi, egitim_etiketleri, batch_size=64, epochs=3, validation_data=(val_verisi, val_etiketleri), callbacks=callback_list)

# Transfer Learning Modelini Değerlendirme
transfer_test_kayip, transfer_test_basari = transfer_model.evaluate(test_verisi, test_etiketleri, verbose=2)
print('Transfer Learning Test kaybı:', transfer_test_kayip)
print('Transfer Learning Test başarısı:', transfer_test_basari)

# Transfer Learning Modeli üzerinde tahminler yapma
transfer_tahminler = transfer_model.predict(test_verisi)
transfer_y_pred = np.argmax(transfer_tahminler, axis=1)
transfer_y_true = np.argmax(test_etiketleri, axis=1)

# Transfer Learning Modeli İçin Sınıflandırma Raporunu Yazdırma
print("Transfer Learning Sınıflandırma Raporu:")
print(classification_report(transfer_y_true, transfer_y_pred))

# Transfer Learning Modeli İçin Karmaşıklık Matrisini Hesaplama
cm_transfer_model = confusion_matrix(transfer_y_true, transfer_y_pred)

# Transfer Learning Modeli İçin Karmaşıklık Matrisini Görselleştirme
plt.figure(figsize=(8, 6))
sns.heatmap(cm_transfer_model, annot=True, cmap='Blues', fmt='.2f', xticklabels=etiket_isimleri, yticklabels=etiket_isimleri)
plt.title('Transfer Learning Karmaşıklık Matrisi')
plt.xlabel('Tahmin Edilen Etiket')
plt.ylabel('Gerçek Etiket')
plt.show()

# Yapay Sinir Ağı Modelini Oluşturma
ann_model = keras.Sequential([
    keras.layers.Flatten(input_shape=(IMG_BOYUTU, IMG_BOYUTU, 3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(num_classes, activation='softmax')
])

# Yapay Sinir Ağı Modelini Derleme
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Yapay Sinir Ağı Modelini Eğitme
ann_gecmis = ann_model.fit(egitim_verisi, egitim_etiketleri, batch_size=64, epochs=3, validation_data=(val_verisi, val_etiketleri), callbacks=callback_list)

# Yapay Sinir Ağı Modelini Değerlendirme
ann_test_kayip, ann_test_basari = ann_model.evaluate(test_verisi, test_etiketleri, verbose=2)
print('Yapay Sinir Ağı Test kaybı:', ann_test_kayip)
print('Yapay Sinir Ağı Test başarısı:', ann_test_basari)

# Yapay Sinir Ağı Modeli üzerinde tahminler yapma
ann_tahminler = ann_model.predict(test_verisi)
ann_y_pred = np.argmax(ann_tahminler, axis=1)
ann_y_true = np.argmax(test_etiketleri, axis=1)

# Yapay Sinir Ağı Modeli İçin Sınıflandırma Raporunu Yazdırma
print("Yapay Sinir Ağı Sınıflandırma Raporu:")
print(classification_report(ann_y_true, ann_y_pred))

# Yapay Sinir Ağı Modeli İçin Karmaşıklık Matrisini Hesaplama
cm_ann_model = confusion_matrix(ann_y_true, ann_y_pred)

# Yapay Sinir Ağı Modeli İçin Karmaşıklık Matrisini Görselleştirme
plt.figure(figsize=(8, 6))
sns.heatmap(cm_ann_model, annot=True, cmap='Blues', fmt='.2f', xticklabels=etiket_isimleri, yticklabels=etiket_isimleri)
plt.title('Yapay Sinir Ağı Karmaşıklık Matrisi')
plt.xlabel('Tahmin Edilen Etiket')
plt.ylabel('Gerçek Etiket')
plt.show()

bitis_zamani = time.time()
gecen_zaman = bitis_zamani - baslangic_zamani
print(f"Eğitim süresi {gecen_zaman} saniye sürdü.")
